<a href="https://colab.research.google.com/github/arimbawa/Aplikasi-IoT/blob/main/Minggu_6_Visualisasi_Data_IoT_dengan_Flask_%26_Chart_js.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minggu ke-6: Visualisasi Data IoT dengan Flask & Chart.js

**Tujuan Pembelajaran**

1. Mahasiswa dapat membuat dashboard berbasis web untuk menampilkan data IoT.
2. Mahasiswa dapat menghubungkan database (Mongita) dengan visualisasi data secara real-time.
3. Mahasiswa memahami penggunaan Flask + Jinja2 Template untuk render data.
4. Mahasiswa dapat menggunakan Chart.js untuk menampilkan grafik IoT (temperature & humidity).

# Materi Teori

* Dashboard IoT: Menampilkan data sensor agar mudah dipahami.
* Flask Template Rendering (Jinja2): Menyajikan data dari backend ke frontend.
* Chart.js: Library JavaScript populer untuk membuat grafik interaktif.
* Update Data via API: Data baru bisa diambil lewat REST API (dari minggu sebelumnya).

# Langkah Praktik di Google Colab

# 1. Install & Import Modul

In [1]:
!pip install flask-ngrok pyngrok mongita flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 30.1 MB/s eta 0:00:00
  Created wheel for mongita: filename=mongita-1.2.0-py3-none-any.whl size=61258 sha256=25481d6b150100edd9cc4b1f1d798cca83548748b7f0b96d6bf2adbb522850ab
  Stored in directory: /root/.cache/pip/wheels/c0/bf/0d/6a2a625c704f15045f2e0286f8a819f2104eeaefb92cd42a4f
Successfully built mongita


# 2. Setup pyngrok
Masukkan token ngrok yang didapatkan dari https://ngrok.com/.

In [2]:
from pyngrok import ngrok

NGROK_TOKEN = "xxxxx"  # Token
ngrok.set_auth_token(NGROK_TOKEN)

# 3. Setup Flask + Database Mongita

In [9]:
from flask import Flask, jsonify, request, render_template_string
from mongita import MongitaClientDisk
from pyngrok import ngrok

# Setup Flask
app = Flask(__name__)

# Setup database
client = MongitaClientDisk()
db = client.iot
collection = db.sensors

# Clear the collection each time the server runs
# Use delete_many as drop() is not implemented in mongita
collection.delete_many({})


# Template HTML dengan Chart.js
template = """
<!DOCTYPE html>
<html>
<head>
    <title>IoT Dashboard</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
</head>
<body>
    <h2>Dashboard IoT</h2>
    <canvas id="myChart" width="400" height="200"></canvas>
    <script>
        async function loadData() {
            let response = await fetch('/data');
            let data = await response.json();

            let labels = data.map(item => item._id);
            let temp = data.map(item => item.temperature);
            let hum = data.map(item => item.humidity);

            const ctx = document.getElementById('myChart').getContext('2d');
            new Chart(ctx, {
                type: 'line',
                data: {
                    labels: labels,
                    datasets: [
                        { label: 'Temperature', data: temp, borderColor: 'red', fill: false },
                        { label: 'Humidity', data: hum, borderColor: 'blue', fill: false }
                    ]
                },
                options: {
                    responsive: true,
                    scales: {
                        y: { beginAtZero: true }
                    }
                }
            });
        }
        loadData();
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(template)

@app.route('/data')
def data():
    docs = list(collection.find())
    return jsonify(docs)

# Endpoint tambah data sensor manual (simulasi)
@app.route('/add', methods=['POST'])
def add():
    data = request.json
    collection.insert_one(data)
    return jsonify({"status": "success"}), 201

# Jalankan ngrok
public_url = ngrok.connect(5000)
print(" * Ngrok URL:", public_url)

app.run(port=5000)

 * Ngrok URL: NgrokTunnel: "https://bdd62916a4f0.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 14:06:43] "POST /add HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 14:06:46] "POST /add HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 14:06:48] "POST /add HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 14:06:50] "POST /add HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 14:06:52] "POST /add HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 14:06:54] "POST /add HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 14:06:56] "POST /add HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 14:06:59] "POST /add HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 14:07:01] "POST /add HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 14:07:03] "POST /add HTTP/1.1" 201 -
INFO:werkzeug:12

# Tugas Minggu ke-6

1. Tambahkan fitur auto-refresh grafik (misalnya refresh setiap 10 detik dengan setInterval).
2. Simulasikan pengiriman 10 data sensor random ke database, lalu tampilkan pada grafik.
3. Buat 2 grafik terpisah: (1) temperature, (2) humidity.